## Delay Theorem (2)
An objective to minimize the cost for delay can be considered to induce a complete order upon two separation-identical aircraft $i$ and $j$ where
$b_i \leq b_j$ and $r_i \leq r_j$.



In [48]:
from z3 import *
import itertools

s = Solver()


Then, we define the following:
- $\delta \in \mathbb{Z} \to \mathbb{Z} \to \mathbb{R}$ (separation-matrix)
- $r_i \in \mathbb{Z} \to \mathbb{R}$ (release time)
- $b_i \in \mathbb{Z} \to \mathbb{R}$ (base time)

then
1. Non-negativity of release times / $r_x$:
$$ r_x \geq 0 \quad \forall x \in S $$

2. Non-negativity of base times / $b_i$
$$ b_x \geq 0 \quad \forall x \in S $$

3. Non-negativity of $\delta$
$$ \delta(x, y) \geq 0 \quad \forall x \in (S \times S) $$

4. Identical separations between `a` and `b` $\delta_a = \delta_b$
$$ \delta(a, x) = \delta(b, x) \wedge \delta(x, a) = \delta(x, b) \quad \forall x \in S $$


In [49]:

δ = Function('δ', IntSort(), IntSort(), RealSort())
R = Function('R', IntSort(), RealSort())
B = Function('B', IntSort(), RealSort())

# --- aircraft etc as before ---
p1, a, p2, b, p3 = Ints('p1 a p2 b p3')
ac = [p1, a, p2, b, p3]

s.add(ForAll([a], R(a) >= 0))
s.add(ForAll([a], B(a) >= 0))
s.add(ForAll([a, b], δ(a, b) >= 0))

# δ-identical a/b
for x_ in ac:
    s.add(δ(a, x_) == δ(b, x_))
    s.add(δ(x_, a) == δ(x_, b))



In the original objective function, costs are raised to an equity variable $\alpha$, which is conflicts with the decidability of SMT logic. Instead, we pose $\alpha$ as an uninterpreted, strictly monotonic function, filling the same role as the original exponentiation but in an SMT decidable way.

$$ x \leq y \implies \alpha (x) < \alpha (y) \quad \forall x,\ y \in \mathbb{R} $$
$$ x \geq 0 \implies \alpha (x) \geq 0 \quad \forall x \in \mathbb{R} $$
$$ α(0) == 1 $$


In [50]:

α = Function('f', RealSort(), RealSort())

# Monotonic and non-negative
x, y = Reals('x y')
s.add(ForAll([x, y], Implies(x <= y, α(x) < α(y))))
s.add(ForAll(x, Implies(x >= 0, α(x) >= 0)))
s.add(α(0) == 1)

Then define some helper functions to encode a $max$ function symbolically. Define a recursive function to compute a symbolic takeoff time for a given sequence.

In [51]:
# zmax and T computation
def zmax(x, y): return If(x >= y, x, y)
def zmax_list(xs):
    m = xs[0]
    for x in xs[1:]:
        m = If(x >= m, x, m)
    return m

def compute_T(seq):
    T = {seq[0]: R(seq[0])}
    for i in range(1, len(seq)):
        preds = seq[:i]
        T[seq[i]] = zmax(R(seq[i]), zmax_list([T[x] + δ(x, seq[i]) for x in preds]))
    return T


Define sequences where $a$ preceeds $b$ and $b$ preceeds $a$

In [52]:

S1, S2 = [p1, a, p2, b, p3], [p1, b, p2, a, p3]
T1, T2 = compute_T(S1), compute_T(S2)

# Delay cost using monotonic f
W1 = Real('W1')
s.add(W1 > 0)
def delay_cost(T):
    return Sum([W1 * α(T[x] - B(x)) for x in [a, b]])

D1, D2 = delay_cost(T1), delay_cost(T2)
s.add(R(a) <= R(b), B(a) <= B(b))
s.add(D1 > D2)

print(s.check())
if s.check() == sat:
    print("\nCounterexample found:\n")
    m = s.model()
    for d in m.decls():
        print(f"{d.name()} = {m[d]}")
else:
    print("\nUNSAT — theorem holds (no counterexample found)\n")

unsat

UNSAT — theorem holds (no counterexample found)

